In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math

from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from pandas.io.json import json_normalize
import json
import gc
gc.enable()
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
def load_df(csv_path='/home/baitong/pywork/RevenuePrediction/all/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
#     print(df.head())
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df


In [2]:
%%time
##读取数据
train_df = load_df()
test_df = load_df("/home/baitong/pywork/RevenuePrediction/all/test.csv")

Loaded train.csv. Shape: (903653, 55)
Loaded test.csv. Shape: (804684, 53)
CPU times: user 3min 49s, sys: 4.12 s, total: 3min 53s
Wall time: 3min 53s


In [2]:
# train_df.to_csv('/home/baitong/pywork/RevenuePrediction/all/train_df.csv')
# test_df.to_csv('/home/baitong/pywork/RevenuePrediction/all/test_df.csv')

In [3]:
#All functions

#FUNCTION FOR PROVIDING FEATURE SUMMARY
def feature_summary(df_fa):
    print('DataFrame shape')
    print('rows:',df_fa.shape[0])
    print('cols:',df_fa.shape[1])
    col_list=['Null','Unique_Count','Data_type','Max/Min','Mean','Std','Skewness','Sample_values']
    df=pd.DataFrame(index=df_fa.columns,columns=col_list)
    df['Null']=list([len(df_fa[col][df_fa[col].isnull()]) for i,col in enumerate(df_fa.columns)])
    #df['%_Null']=list([len(df_fa[col][df_fa[col].isnull()])/df_fa.shape[0]*100 for i,col in enumerate(df_fa.columns)])
    df['Unique_Count']=list([len(df_fa[col].unique()) for i,col in enumerate(df_fa.columns)])
    df['Data_type']=list([df_fa[col].dtype for i,col in enumerate(df_fa.columns)])
    for i,col in enumerate(df_fa.columns):
        if 'float' in str(df_fa[col].dtype) or 'int' in str(df_fa[col].dtype):
            df.at[col,'Max/Min']=str(round(df_fa[col].max(),2))+'/'+str(round(df_fa[col].min(),2))
            df.at[col,'Mean']=df_fa[col].mean()
            df.at[col,'Std']=df_fa[col].std()
            df.at[col,'Skewness']=df_fa[col].skew()
        df.at[col,'Sample_values']=list(df_fa[col].unique())
           
    return(df.fillna('-'))

#FUNCTION FOR READING DICTIONARY ITEMS AND HANDLING KEYERROR
def get_val(x,col):
    try:
        y=x[col]
    except:
        y=np.nan
    return(y)

#FUNCTION FOR CALCULATING RSME
def rsme(y,pred):
    return(mean_squared_error(y,pred)**0.5)

In [4]:
#获取不变的常量列，模型无法在常量数据计学到东西，数据与处理时需要drop
const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ]
train_df = train_df.drop(const_cols , axis=1)
test_df = test_df.drop(const_cols, axis=1)
####删除test中不存在的列
train_df = train_df.drop(["trafficSource.campaignCode"], axis=1)
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')

In [5]:
#COMBINING TRAIN AND TEST DATASET
df_combi=pd.concat([train_df,test_df],ignore_index=True)
train_df['revenue_status']=df_combi['totals.transactionRevenue'].apply(lambda x: 0 if x==0 else 1)

In [6]:
#EXTRACTING DAY_OF_WEEK, HOUR, DAY, MONTH FROM DATE 
df_combi['date'] = pd.to_datetime(df_combi['visitStartTime'], unit='s')
df_combi['day_of_week'] = df_combi['date'].dt.dayofweek
df_combi['hour'] = df_combi['date'].dt.hour
df_combi['day'] = df_combi['date'].dt.day
df_combi['month'] = df_combi['date'].dt.month

#ADDING ANOTHER FEATURE revenue_status TO INDICATE PRESENCE/ABSENCE OF REVENUE FOR EACH OBSERVATION
df_combi['revenue_status']=df_combi['totals.transactionRevenue'].apply(lambda x: 0 if x==0 else 1)

In [7]:
%%time
#CONVERTING ALL THE STRINGS IN CATEGORICAL FEATURES TO LOWER CASE
for col in df_combi.columns:
    if ((df_combi[col].dtype=='object') & (col!='fullVisitorId')):
        df_combi[col]=df_combi[col].apply(lambda x:str(x).lower())
        
#REPLACING STRING 'nan' WITH np.nan
df_combi.replace('nan',np.nan,inplace=True)

CPU times: user 32.4 s, sys: 704 ms, total: 33.1 s
Wall time: 33.1 s


In [8]:
%%time
#CONVERTING CATEGORICAL FEATURES (LESS THAN 10 UNIQUE VALUES) TO DUMMIES
df_combi.drop(['device.isMobile'],axis=1,inplace=True)

cat_col=['channelGrouping','device.deviceCategory','trafficSource.adwordsClickInfo.slot',
         'trafficSource.adwordsClickInfo.adNetworkType',
         'trafficSource.adwordsClickInfo.isVideoAd','trafficSource.medium',
        'geoNetwork.continent']

dummy=pd.DataFrame()
col_name = ['channelGrouping','deviceCategory','tsadwordsClickInfo_slot',
                  'tsadwordsClickInfo_adNetworkType',
                   'tsadwordsClickInfo_isVideoAd','tsmedium',
            'geoNetwork_continent']

for col,name in zip(cat_col,col_name):
    dummy=pd.concat([dummy,pd.get_dummies(train_df[col],prefix=name)],axis=1)


print('Newly created dummy cols:',len(dummy.columns))
df_combi=pd.concat([df_combi,dummy],axis=1)

df_combi.drop(cat_col,axis=1,inplace=True)

Newly created dummy cols: 29
CPU times: user 4.81 s, sys: 1.01 s, total: 5.82 s
Wall time: 5.81 s


In [9]:
#SOME BASIC DATA CLEANUP
df_combi['totals.newVisits'].fillna(0,inplace=True) 
df_combi['totals.bounces'].fillna(0,inplace=True)
df_combi['trafficSource.adwordsClickInfo.page'].fillna(0,inplace=True)
df_combi['trafficSource.isTrueDirect'].replace({np.nan:0,'true':1},inplace=True)

In [10]:
df_combi.head()

,date,device.browser,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,...,tsmedium_cpc,tsmedium_cpm,tsmedium_organic,tsmedium_referral,geoNetwork_continent_(not set),geoNetwork_continent_Africa,geoNetwork_continent_Americas,geoNetwork_continent_Asia,geoNetwork_continent_Europe,geoNetwork_continent_Oceania
0,2016-09-02 15:33:05,chrome,windows,1131660440785968503,izmir,turkey,(not set),ttnet.com.tr,izmir,western asia,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2016-09-03 05:22:27,firefox,macintosh,377306020877927890,not available in demo dataset,australia,not available in demo dataset,dodo.net.au,not available in demo dataset,australasia,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2016-09-03 01:16:26,chrome,windows,3895546263509774583,madrid,spain,(not set),unknown.unknown,community of madrid,southern europe,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2016-09-03 05:40:13,uc browser,linux,4763447161404445595,not available in demo dataset,indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,southeast asia,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2016-09-02 13:23:20,chrome,android,27294437909732085,not available in demo dataset,united kingdom,not available in demo dataset,unknown.unknown,not available in demo dataset,northern europe,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
%%time
#GENERATING RANKS FOR CATEGORICAL FEATURES WITH UNIQUE VALUES GREATER THAN 10
#RANKS ARE GENERATED USING REVENUE PERCENTAGE
cols=[x for x in df_combi.columns if x not in ['fullVisitorId','sessionId','geoNetwork.networkDomain','trafficSource.adwordsClickInfo.gclId']]

for col in cols:
    if df_combi[col].dtype=='object':
        df_combi[col].fillna('others',inplace=True)
        col_list=['revenue_status','totals.transactionRevenue']
        col_list.append(col)
        print(col_list)
        df=df_combi[col_list].groupby(col).aggregate({col:['count'],'revenue_status':['sum'],'totals.transactionRevenue':['sum']}).reset_index()
        df.columns=[col,col+"_count",'revenue_status_sum','totals.transactionRevenue_sum']
        df['revenue_perc']=df['totals.transactionRevenue_sum']/df[col+"_count"]
        df['rank']=df['revenue_perc'].rank(ascending=1)
        
        replace_dict={}
        final_dict={}
        for k,col_val in enumerate(df[col].values):
            replace_dict[col_val]=df.iloc[k,5]
        final_dict[col]=replace_dict
        df_combi.replace(final_dict,inplace=True)
        del df,replace_dict,final_dict
        gc.collect()

['revenue_status', 'totals.transactionRevenue', 'device.browser']
['revenue_status', 'totals.transactionRevenue', 'device.operatingSystem']
['revenue_status', 'totals.transactionRevenue', 'geoNetwork.city']
['revenue_status', 'totals.transactionRevenue', 'geoNetwork.country']
['revenue_status', 'totals.transactionRevenue', 'geoNetwork.metro']
['revenue_status', 'totals.transactionRevenue', 'geoNetwork.region']
['revenue_status', 'totals.transactionRevenue', 'geoNetwork.subContinent']
['revenue_status', 'totals.transactionRevenue', 'totals.bounces']
['revenue_status', 'totals.transactionRevenue', 'totals.hits']
['revenue_status', 'totals.transactionRevenue', 'totals.newVisits']
['revenue_status', 'totals.transactionRevenue', 'totals.pageviews']
['revenue_status', 'totals.transactionRevenue', 'trafficSource.adContent']
['revenue_status', 'totals.transactionRevenue', 'trafficSource.adwordsClickInfo.page']
['revenue_status', 'totals.transactionRevenue', 'trafficSource.campaign']
['revenue_

In [12]:
#将字符串转换为数字
df_combi['geoNetwork.networkDomain'],unique=pd.factorize(df_combi['geoNetwork.networkDomain'])
df_combi['trafficSource.adwordsClickInfo.gclId'],unique1=pd.factorize(df_combi['trafficSource.adwordsClickInfo.gclId'])

In [13]:
#SPLITING COMBINED DATASET BACK TO TRAIN AND TEST SETS
train=df_combi[:len(train_df)]
test=df_combi[len(train_df):]

In [16]:
print('Train shape:',train.shape,' Test shape:',test.shape)

Train shape: (903653, 61)  Test shape: (804684, 61)


In [18]:
X=train.drop(['sessionId','visitId','date','geoNetwork.networkDomain','trafficSource.adwordsClickInfo.gclId'],axis=1)
X_test=test.drop(['sessionId','visitId','date','geoNetwork.networkDomain','trafficSource.adwordsClickInfo.gclId'],axis=1)   

In [22]:
X_test.reset_index(drop=True,inplace=True)
# X_test.head()

In [23]:
%%time
agg_func={}
agg_col=['fullVisitorId']
for col in [x for x in X.columns if x not in ['fullVisitorId']]:
    if col=='totals.transactionRevenue':
        agg_func[col]=['sum']
        agg_col.append(str(col)+'_sum')
    elif col=='revenue_status':
        agg_func[col]=['sum']
        agg_col.append(str(col)+'_sum')
    else:
        agg_func[col]=['sum','max','min','mean','var','std']
        agg_col.append(str(col)+'_sum')
        agg_col.append(str(col)+'_max')
        agg_col.append(str(col)+'_min')
        agg_col.append(str(col)+'_mean')
        agg_col.append(str(col)+'_var')
        agg_col.append(str(col)+'_std')
    
X=X.groupby(X.fullVisitorId).aggregate(agg_func).reset_index()
X.columns=agg_col

X_test=X_test.groupby(X_test.fullVisitorId).aggregate(agg_func).reset_index()
X_test.columns=agg_col

CPU times: user 31.5 s, sys: 5.75 s, total: 37.2 s
Wall time: 48.7 s


In [26]:
%%time

#CREATING y_dummy FOR USING STRATIFIED KFOLD
y_dummy=X.revenue_status_sum.apply(lambda x: 0 if x==0 else 1)

#TARGET FEATURE CONVERTED TO NATURAL LOG
# y=pd.Series(X['totals_transactionRevenue_sum'])
y=X['totals.transactionRevenue_sum'].apply(lambda x: np.log1p(x))

#PEPARING DATA FOR TRAINING LGBM MODEL
X=X.drop(['totals.transactionRevenue_sum','fullVisitorId','revenue_status_sum'],axis=1)

#FINAL DATAFRAME FOR SUBMISSION
col=['fullVisitorId','totals.transactionRevenue_sum']
final=X_test[col] 
final.columns=['fullVisitorId','PredictedLogRevenue']

#FINAL TEST FEATURES USED FOR PREDICTING SUBMISSION
X_test=X_test.drop(['fullVisitorId','totals.transactionRevenue_sum','revenue_status_sum'],axis=1)

CPU times: user 1.45 s, sys: 844 ms, total: 2.3 s
Wall time: 2.33 s


In [45]:
X.to_csv('/home/baitong/pywork/RevenuePrediction/all/X.csv')

In [42]:
X.describe()

,device.browser_sum,device.browser_max,device.browser_min,device.browser_mean,device.browser_var,device.browser_std,device.operatingSystem_sum,device.operatingSystem_max,device.operatingSystem_min,device.operatingSystem_mean,...,geoNetwork_continent_Europe_min,geoNetwork_continent_Europe_mean,geoNetwork_continent_Europe_var,geoNetwork_continent_Europe_std,geoNetwork_continent_Oceania_sum,geoNetwork_continent_Oceania_max,geoNetwork_continent_Oceania_min,geoNetwork_continent_Oceania_mean,geoNetwork_continent_Oceania_var,geoNetwork_continent_Oceania_std
count,714167.000000,714167.000000,714167.000000,714167.000000,93492.000000,93492.000000,714167.000000,714167.000000,714167.000000,714167.000000,...,714167.000000,714167.000000,93492.000000,93492.000000,714167.000000,714167.000000,714167.000000,714167.000000,93492.000000,93492.000000
mean,159.735469,125.977671,125.975486,125.976625,0.301216,0.010555,26.870152,21.168844,21.168020,21.168428,...,0.234428,0.234785,0.002068,0.003390,0.021079,0.018064,0.017972,0.018018,0.000251,0.000412
std,183.204769,8.910949,8.916015,8.912234,24.074726,0.548733,31.049459,1.715610,1.715748,1.715508,...,0.423641,0.423672,0.028952,0.045347,0.183206,0.133184,0.132850,0.132943,0.010082,0.015839
min,60.500000,60.500000,60.500000,60.500000,0.000000,0.000000,9.000000,9.000000,9.000000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,126.000000,125.000000,125.000000,125.000000,0.000000,0.000000,21.000000,20.000000,20.000000,20.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,128.000000,128.000000,128.000000,128.000000,0.000000,0.000000,21.000000,21.000000,21.000000,21.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,128.000000,128.000000,128.000000,128.000000,0.000000,0.000000,23.000000,23.000000,23.000000,23.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,35862.000000,129.000000,129.000000,129.000000,2346.125000,48.436815,5838.000000,24.000000,24.000000,24.000000,...,1.000000,1.000000,0.500000,0.707107,31.000000,1.000000,1.000000,1.000000,0.500000,0.707107


In [29]:
%%time
#LGBMRegressor. THIS REQUIRES FURTHER PARAMETER TUNINIG
model=LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=-1,learning_rate=0.01,n_estimators=1000,max_bin=255,subsample_for_bin=50000,
              objective=None,min_split_gain=0,min_child_weight=3,min_child_samples=10,subsample=1,subsample_freq=1,colsample_bytree=1,
              reg_alpha=0.1,reg_lambda=0,seed=17,silent=False,nthread=-1,n_jobs=-1)


k=1
splits=5
avg_score=0


skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=200)
print('\nStarting KFold iterations...')
for train_index,test_index in skf.split(X,y_dummy):
    df_X=X.iloc[train_index,:]
    df_y=y.iloc[train_index]
    val_X=X.iloc[test_index,:]
    val_y=y.iloc[test_index]

    model.fit(df_X,df_y)

    preds_x=pd.Series(model.predict(val_X))
    acc=rsme(val_y,preds_x)
    print('Iteration:',k,'  rmse:',acc)
    
    if k==1:
        score=acc
        model1=model
        preds=pd.Series(model.predict(X_test))
        
    else:
        preds1=pd.Series(model.predict(X_test))
        preds=preds+preds1
        if score>acc:
            score=acc
            model1=model
    avg_score=avg_score+acc        
    k=k+1
print('\n Best score:',score,' Avg Score:',avg_score/splits)
preds=preds/splits


Starting KFold iterations...
Iteration: 1   rmse: 1.6080565159754454
Iteration: 2   rmse: 1.5907234230418628
Iteration: 3   rmse: 1.6021121036886379
Iteration: 4   rmse: 1.5891252730321044
Iteration: 5   rmse: 1.5920087758251273

 Best score: 1.5891252730321044  Avg Score: 1.5964052183126358
CPU times: user 57min 53s, sys: 28.6 s, total: 58min 22s
Wall time: 6min 36s


In [30]:
#PREPARING PREDICTED DATA
final['PredictedLogRevenue']=pd.Series(preds)
#GROUPING PREDICTED DATA ON fullVisitorId
final = final.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
final.columns = ["fullVisitorId", "PredictedLogRevenue"]

In [32]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617242 entries, 0 to 617241
Data columns (total 2 columns):
fullVisitorId          617242 non-null object
PredictedLogRevenue    617242 non-null float64
dtypes: float64(1), object(1)
memory usage: 9.4+ MB


In [37]:
submission=pd.read_csv('/home/baitong/pywork/RevenuePrediction/all/sample_submission.csv')

#CREATING JOIN BETWEEN PREDICTED DATA WITH SUBMISSION FILE
submission=submission.join(final.set_index('fullVisitorId'),on='fullVisitorId',lsuffix='_sub')
submission.drop('PredictedLogRevenue_sub',axis=1,inplace=True)

#HANDLING NaN IN CASE OF MISSING fullVisitorId
submission.fillna(0,inplace=True)

#SUBMITING FILE
submission.to_csv('/home/baitong/pywork/RevenuePrediction/submission.csv',index=False)